# 中文分词技术
中文自动分词技术主要可以归纳为三种：规则分词，统计分词，混合分词。
## 规则分词

基于规则的分词是一种机械分词方法，主要是通过维护词典，在切分语句时，将语句中的每个词和词表中的词逐一进行匹配，找到则切分，否者不予切分。这种方式需要通过人工建立词库，并不断更新词库，否则对于新时代出现的词将无法识别。
按照切分的方式，主要有正向最大匹配法、逆向最大匹配法、双向最大匹配法。
## 正向最大匹配法
正向最大匹配法（Maximum Match Method，简称MM法）的基本思想是：假定分词词典里的最长词的长度为k，那么就用待处理文档的当前长度为k的字符串和词典进行匹配，如果找到则将分词，否则去掉最后一个字符在进行匹配。

In [195]:
class MM:
    def __init__(self, corpus):
        self.dict = corpus
        self.max_len = self.get_max_len()

    def add_word(self, word):
        # 词表扩充
        self.dict.add(word)
        if len(word) > self.max_len:
            self.max_len = len(word)
        
    
    def get_max_len(self):
        max_len = 0
        for word in self.dict:
            if len(word) > max_len:
                max_len = len(word)
        return max_len

    def tokenize(self, doc):
        doc_len = len(doc)
        # 下标0开始
        start_index = 0
        tokens = []
        while start_index < doc_len:
            # 当前最长的字符串候选
            end_index = start_index +  min(doc_len - start_index, self.max_len)
            while end_index > start_index:
                span = doc[start_index: end_index]
                if span in self.dict:
                    tokens.append(span)
                    start_index = end_index
                    break
                
                end_index -= 1
            
        return tokens

In [196]:
corpus = {"研究","研究生","生命","命","的","起源"}

mm = MM(corpus)

text = "研究生命的起源"
mm.tokenize(text)

['研究生', '命', '的', '起源']

## 逆向最大匹配法
逆向最大匹配法（Reverse Maximum Match Method，简称RMM法）的基本思想是和正向最大匹配基本一致，只是顺序是从文档的末端开始向前扫描。\
统计结果表明，单纯使用正向最大匹配的错误率为1/169，而逆向最大匹配是1/245，这是由于汉语中偏正结构较多，从后向前匹配可以适当提高精确度。

In [197]:
class RMM:
    def __init__(self, corpus):
        self.dict = corpus
        self.max_len = self.get_max_len()

    def add_word(self, word):
        # 词表扩充
        self.dict.add(word)
        if len(word) > self.max_len:
            self.max_len = len(word)
        
    
    def get_max_len(self):
        max_len = 0
        for word in self.dict:
            if len(word) > max_len:
                max_len = len(word)
        return max_len

    def tokenize(self, doc):
        doc_len = len(doc)
        # 下标0开始
        end_index = doc_len
        tokens = []
        while end_index > 0:
            # 当前最长的字符串候选
            start_index = end_index - min(end_index, self.max_len)
            
            while start_index < end_index:
                span = doc[start_index: end_index]
                # print(span)
                if span in self.dict:
                    tokens.append(span)
                    end_index = start_index
                    break
                
                start_index += 1
            
        return tokens[::-1]

In [198]:
corpus = {"研究","研究生","生命","命","的","起源"}

rmm = RMM(corpus)

text = "研究生命的起源"
rmm.tokenize(text)

['研究', '生命', '的', '起源']

## 双向最大匹配法
双向最大匹配法(Bi-direction Matching method)是将正向最大匹配和逆向最大匹配的结果进行融合比较得到的。\
经过SunM.S和Benjamin K.T研究表明: 
- 90%的中文句子，使用正向和逆向最大匹配完全重合且正确。
- 大概9%的中文句子这两种切分的结果不一样，但必定有一个是正确的。
- 只有1%的中文句子两种切分都失败。

双向最大匹配的原则是：
- (1)如果正反向分词结果词数不同，则取分词数量较少的那个(上例:“南京市/长江/大桥”的分词数量为3而“南京市/长江大桥”的分词数量为2，所以返回分词数量为2的)。
- (2)如果分词结果词数相同:
    - 1)分词结果相同，就说明没有歧义，可返回任意一个。
    - 2)分词结果不同，返回其中单字较少的那个。比如:上述示例代码中，正向最大匹配返回的结果为“['研究生----'，' 命 ----','的 ----','起源 ----']”，其中单字个数为2个;而逆向最大匹配返回的结果为“['研究 ----'，'生命 ----','的 ----','起源----']”，其中单字个数为1个。所以返回的是逆向最大匹配的结果。



In [199]:
class BIMM:
    def __init__(self, corpus):
        self.dict = corpus
        self.mm = MM(corpus)
        self.rmm = RMM(corpus)
        

    def add_word(self, word):
        # 词表扩充
        self.mm.add_word(word)
        self.rmm.add_word(word)

    def tokenize(self, doc):
        # 正向
        mm_tokens = self.mm.tokenize(doc)
        rmm_tokens = self.rmm.tokenize(doc)

        if len(mm_tokens) < len(rmm_tokens):
            return mm_tokens
        elif len(mm_tokens) > len(rmm_tokens):
            return rmm_tokens
        else:
            mm_count_dict = self.get_count_dict(mm_tokens)
            rmm_count_dict = self.get_count_dict(rmm_tokens)

            count = 1
            while count in mm_count_dict:
                mm_count = mm_count_dict[count]
                rmm_count = rmm_count_dict[count]
                
                if mm_count == rmm_count:
                    count += 1
                elif mm_count > rmm_count:
                    return rmm_tokens
                else:
                    return mm_tokens
            
        return -1

    def get_count_dict(self, tokens):
        from collections import defaultdict
        count_dict = defaultdict(int)
        for token in tokens:
            count_dict[len(token)] += 1
        return count_dict

In [200]:
corpus = {"研究","研究生","生命","命","的","起源"}

bimm = BIMM(corpus)

text = "研究生命的起源"
bimm.tokenize(text)

['研究', '生命', '的', '起源']

# 统计分词
基于规则的分词，一般都较为简单高效，但是词典的维护是一个很庞大的工程。在网络发达的今天，网络新词层出不穷，很难通过词典覆盖到所有词。 \
随着大规模语料库的建立，统计机器学习方法的研究和发展，基于统计的中文分词算法逐渐成为主流。 \
基于统计的分词，一般需要两步操作:
1. 建立统计语言模型
2. 对句子进行单词划分，然后对划分结果进行概率计算，获得概率最大的分词方式。这里需要使用统计学习算法，如隐马尔可夫（HMM）、条件随机场（CRF）等


In [203]:
class HMM:
    def __init__(self):
        import os

        self.model_file = './data/hmm_model.pkl'
        self.state_list = ['B', 'M', 'E', 'S']
        self.load_para = False 

    def try_load_model(self, trained):
        if trained:
            import pickle
            with open(self.model_file, 'rb') as f:
                self.A_dic = pickle.load(f)
                self.B_dic = pickle.load(f)
                self.Pi_dic = pickle.load(f)
                self.load_para = True
        else:
            # 状态转移概率矩阵, p(o_t-1 | o_t)
            self.A_dic = {}
            # 发射概率矩阵, p(lambda_t | o_t)
            self.B_dic = {}
            # 初始概率矩阵
            self.Pi_dic = {}
            self.load_para = False

    def train(self, path):
        self.try_load_model(False)

        # 统计状态出现的次数
        Count_dic = {}

        def init_parameters():
            for state in self.state_list:
                self.A_dic[state] = {s:0 for s in self.state_list} # 初始每个状态之间的转移都为0
                self.Pi_dic[state] = 0
                self.B_dic[state] = {} # 没有单词进行转换

                Count_dic[state] = 0

        def makeLabel(text):
            if len(text) == 1:
                return ['S']
            else:                
                return ['B'] + ['M'] * (len(text) - 2) + ['E']

        init_parameters()

        line_num = -1
        words = set()
        with open(path, encoding='utf-8') as f:
            for line in f:
                # 1. 去空格
                line_num += 1
                line = line.strip()
                if not line:
                    continue

                word_list = [word for word in line if word != " "]
                words |= set(word_list)
                
                line_list = line.split()

                line_state = []
                for word in line_list:
                    line_state.extend(makeLabel(word))
                # print(line_state, word_list)
                assert len(line_state) == len(word_list)

                for k,v in enumerate(line_state):
                    Count_dic[v] += 1
                    if k == 0:
                        self.Pi_dic[v] += 1
                    else:
                        self.A_dic[line_state[k-1]][v] += 1
                        self.B_dic[v][word_list[k]] = self.B_dic[v].get(word_list[k],0) + 1

            self.Pi_dic = {k: v / line_num for k, v in self.Pi_dic.items()}
            self.A_dic = {k: {k1: v1 / Count_dic[k] for k1,v1 in v.items()} for k,v in self.A_dic.items()}
            self.B_dic = {k: {k1: (v1 + 1) / Count_dic[k]  for k1, v1 in v.items()} for k, v in self.B_dic.items()}

            import pickle
            with open(self.model_file, 'wb') as f:
                pickle.dump(self.A_dic, f)
                pickle.dump(self.B_dic, f)
                pickle.dump(self.Pi_dic,f)

                

    def viterbi(self, text, states, start_p, trans_p, emit_p):
        V = [{}]
        path = {}

        for y in states:
            V[0][y] = start_p[y] * emit_p[y].get(text[0],0)
            path[y] = [y]
        

        for t in range(1, len(text)):
            V.append({})
            newpath = {}
            never_seen = text[t] not in emit_p['S'].keys() and \
                text[t] not in emit_p['E'].keys() and \
                text[t] not in emit_p['M'].keys() and \
                text[t] not in emit_p['B'].keys()
            

            for y in states:
                emitP = emit_p[y].get(text[t], 0) if not never_seen else 1.0
        
                (prob, state) = max(
                    [(V[t-1][y0] * trans_p[y0].get(y, 0) * emitP, y0) for y0 in states if V[t-1][y0] > 0]
                )
                V[t][y] = prob
                newpath[y] = path[state] + [y]
            
            path = newpath
        
        if emit_p['M'].get(text[-1], 0) > emit_p['S'].get(text[-1], 0):
            (prob, state) = max([(V[len(text)-1][y], y) for y in ('E', 'M')])
        else:
            (prob, state) = max([(V[len(text)-1][y], y) for y in states])

        return (prob, path[state])

    def cut(self, text):
        import os
        if not self.load_para:
            self.try_load_model(os.path.exists(self.model_file))
        prob, pos_list = self.viterbi(text, self.state_list, self.Pi_dic, self.A_dic, self.B_dic)

        begin, next= 0, 0
        for i, char in enumerate(text):
            pos = pos_list[i]
            if pos == 'B':
                begin = i
            elif pos == 'E':
                yield text[begin: i+1]
                next = i+1
            elif pos == 'S':
                yield char
                next = i+1
        if next < len(text):
            yield text[next:]
    

In [205]:
hmm = HMM()
hmm.train('./data/trainCorpus.txt_utf8.txt')
text = "这是一个非常棒的方案！"
res = hmm.cut(text)
print(str(list(res)))


SyntaxError: incomplete input (3588079757.py, line 5)